# Machine learning using EKF

In [9]:
from IPython import display
import os
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from load import load_abalone_data,load_bikes_data
# Importing Pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from  sklearn.datasets import make_regression
from sklearn.datasets import load_boston
from tqdm import tqdm as tqdm

In [10]:
# Select device which you are going to use for training
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


### Import Data Sets
Testing using a toy sine data

In [11]:
X,y = load_bikes_data()
print(y.shape)
print(X.shape)

(17379,)
(17379, 14)


## Data Partition


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25)
print(X_train.shape)
print(X_test.shape)
# x_train_scaled = scaler.fit_transform(x_train)
# x_test_scaled = scaler.transform(x_test)

(13034, 14)
(4345, 14)


## Define Neural network

In [13]:
class MLP(nn.Module):
    def __init__(self, n_inputs, n_hidden_layer, n_outputs,bias=True):
        super(MLP, self).__init__()
        # YOUR CODE HERE
        #raise NotImplementedError()
        self.fc1 = nn.Linear(n_inputs, n_hidden_layer, bias)
        self.fc2 = nn.Linear(n_hidden_layer, n_hidden_layer, bias)
        self.fc3 = nn.Linear(n_hidden_layer, n_hidden_layer, bias)
        self.fc4 = nn.Linear(n_hidden_layer, n_outputs, bias)
        

    def forward(self, x):
        # YOUR CODE HERE
        #raise NotImplementedError()
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x    

In [14]:
## Testing net
n_inputs = X.shape[1]
n_outputs = 1
n_hidden_layer = 100
test_net = MLP(n_inputs, n_hidden_layer, n_outputs)
print(test_net)

MLP(
  (fc1): Linear(in_features=14, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=1, bias=True)
)


In [15]:
# Define number of Input and Output layers
torch.set_default_dtype(torch.float64)
n_inputs = X.shape[1]
n_outputs = 1
n_hidden_layer = 100
mlp = MLP(n_inputs,n_hidden_layer, n_outputs)
mpl = mlp.to(device)
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.005)
n_epochs = 100

train_accuracy_history = []
test_accuracy_history = []

x_train = torch.tensor(X_train, device=device, dtype=torch.float64)
y_train = torch.tensor(y_train, device=device, dtype=torch.float64)

ceLoss = nn.MSELoss()



for epoch in tqdm(range(n_epochs)):
    
    #Set gradients as zero
    mlp.zero_grad()
    #Calculate outputs with forward()
    outputs = mlp(x_train)   
    
    #calculate loss
    loss = ceLoss(outputs, y_train)
#     print(f"The output is :{outputs.item()} expected is :{y_train}")
    #calculate gradients - calling backward()
    loss.backward()
    
    #update weights using optimizer
    optimizer.step()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:44<00:00,  1.02s/it]


In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
with torch.no_grad():
    x_test = torch.tensor(X_test, device=device, dtype=torch.float64)
    y_pred = mlp.forward(x_test)
    y_pred = y_pred.cpu().data.numpy()
    error = mean_squared_error(y_test,y_pred)
    print(np.sqrt(error))

182.18075146040852
